### Climate Diet Slider ###
This notebook contains the code to create a slider to show the sensitivity of a vegan diet on carbon equivalent emissions. 
The calculations performed assume the world is already 3% vegan, which is a bit of an underestimate because a good portion of the world is vegetarian or at least flexitarian. While this builds some conservatism into the data presented, there is some unconservatism due to the fact that calories are essentially being erased. I would be interested in adding to this a factor for increasing emissions of other areas of the world due to a reduction in dependence on animal calories. However, the point of this is to show the general trend, and just how much is at stake.

The data is from 2018 and can be found here: https://ourworldindata.org/food-ghg-emissions.

In [10]:
import pandas as pd

# Normal diet emissions, where ~3% of the world is already vegan.
normal_pct = pd.Series(
    {
        'Supply Chain': 0.18,
        'Livestock': 0.31,
        'Animal Feed': 0.06,
        'Human Feed': 0.21,
        'Livestock Land Use': 0.16,
        'Human Food Land Use': 0.08
    }, name='Normal Diet Emissions Breakdown'
)

vegan_pct = pd.Series(
    {
        'Supply Chain': 0.18,
        'Livestock': 0,
        'Animal Feed': 0,
        'Human Feed': 0.21,
        'Livestock Land Use': 0,
        'Human Food Land Use': 0.08
    }, name='WFPB Diet Emissions Breakdown'
)

df = pd.DataFrame([normal_pct, vegan_pct]).T

# calculate the percent difference between the two diets
df['Percent Difference'] = (df['WFPB Diet Emissions Breakdown'] - df['Normal Diet Emissions Breakdown']) / df['Normal Diet Emissions Breakdown']

# convert the normal diet to MMT CO2e
PCT_FOOD_EMISSIONS_2018 = 0.26
KG_CO2E_GLOBAL_EMISSIONS_2018 = 52300 * 10**6 * 10**3
df['Normal Diet Emissions'] = df['Normal Diet Emissions Breakdown'] * PCT_FOOD_EMISSIONS_2018 * MMT_CO2E_GLOBAL_EMISSIONS_2018

# divide by the population of the world in 2018
df['Normal Diet Emissions'] = df['Normal Diet Emissions'] / (7.6 * 10**9)

# Based on the percent difference, calculate the vegan diet emissions
df['WFPB Diet Emissions'] = df['Normal Diet Emissions'] * (1 + df['Percent Difference'])

# Calculate the emissions of an 80% vegan diet
df['80% Vegan Diet Emissions'] = df['Normal Diet Emissions'] * (1 + df['Percent Difference'] * 0.8)

# Create a new dataframe with the emissions data ranging from 0 to 80% vegan, in steps of 1%
slider_df = pd.DataFrame(index=pd.Series(range(0, 81), name='Percent Vegan'))
slider_df['Supply Chain'] = df['Normal Diet Emissions']['Supply Chain'] * (1 + df['Percent Difference']['Supply Chain'] * (slider_df.index / 100))
slider_df['Livestock'] = df['Normal Diet Emissions']['Livestock'] * (1 + df['Percent Difference']['Livestock'] * (slider_df.index / 100))
slider_df['Animal Feed'] = df['Normal Diet Emissions']['Animal Feed'] * (1 + df['Percent Difference']['Animal Feed'] * (slider_df.index / 100))
slider_df['Human Feed'] = df['Normal Diet Emissions']['Human Feed'] * (1 + df['Percent Difference']['Human Feed'] * (slider_df.index / 100))
slider_df['Livestock Land Use'] = df['Normal Diet Emissions']['Livestock Land Use'] * (1 + df['Percent Difference']['Livestock Land Use'] * (slider_df.index / 100))
slider_df['Human Food Land Use'] = df['Normal Diet Emissions']['Human Food Land Use'] * (1 + df['Percent Difference']['Human Food Land Use'] * (slider_df.index / 100))

# Offset the data so that the first index corresponds to 3% vegan
slider_df.index = slider_df.index + 3

# Truncate the data at 80% vegan
slider_df = slider_df[:80]

# Transform the data into a format that can be used by the slider
slider_df = slider_df.stack().reset_index()
slider_df.columns = ['Percent Vegan', 'Emission Type', 'Emissions']


In [49]:
import altair as alt
import datapane as dp

# Create a slider chart where you can select the percent vegan (the slider_df index)
slider = alt.binding_range(min=3, max=80, step=1, name='Percent Whole Foods Plant Based Diets')
select_percent_vegan = alt.selection_single(name='Select', fields=['Percent Vegan'], bind=slider, init={'Percent Vegan': 3})

# plot slider_df where the x axis is percent vegan and the y axis is the emissions for each category
diet_chart = alt.Chart(slider_df).mark_bar(size=100).encode(
    x=alt.X('Percent Vegan:Q', axis=alt.Axis(title='Percent Whole Food Plant Based Diets'), scale=alt.Scale(zero=False)),
    y=alt.Y('sum(Emissions):Q', axis=alt.Axis(title='Yearly Emissions (kg CO₂e)'), scale=alt.Scale(domain=(0, 1800))),
    color=alt.Color('Emission Type', legend=alt.Legend(title='Emission Type')),
    order=alt.Order('Emissions:O', sort='descending'),
    tooltip=[
        'Emission Type',
        alt.Tooltip('Emissions', format='.0f', title='Yearly Emissions (kg CO₂e)'),
    ]
).properties(
    width=400,
    height=400,
    title={
            'text': ['Food Emissions of the Average Person'],
            'subtitle': ['Varying Percent of the World following a WFPB diet'],
            'subtitleColor': 'gray',
           }
).add_selection(
    select_percent_vegan
).transform_filter(
    select_percent_vegan
)

# Upload the chart to datapane
dp.upload_report(
    [
        dp.Plot(diet_chart.resolve_scale(color='independent')),
    ],
    name='ClimateDietSlider',
    visibility='PUBLIC',
    responsive=False,
    scale_factor=2,
)

Uploading report and associated data - *please wait...*

Report successfully uploaded - view and share at <a href='https://cloud.datapane.com/reports/VkBXDP3/climatedietslider/' target='_blank'>here</a>.

Uploaded Report - view at https://cloud.datapane.com/reports/VkBXDP3/climatedietslider/